In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84
INFO     Registered line magic 'reload_kedro_mlflow'                                                                     __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pasteur.metadata import Metadata

trn: pd.DataFrame = catalog.load("mimic_tab_admissions.trn.table")
table = catalog.load("mimic_tab_admissions.wrk.idx_table")

In [ ]:
attrs = trn["idx"].get_attributes()

In [ ]:
from pasteur.transform.attribute import LevelColumn

col: LevelColumn = attrs["dischtime"]["dischtime_day"]
col

Idx['0.4921875', '1.4765625', '2.4609375', '3.4453125', '4.4296875', '5.4140625', '6.3984375', '7.3828125', '8.3671875', '9.3515625', '10.3359375', '11.3203125', '12.3046875', '13.2890625', '14.2734375', '15.2578125', '16.242188', '17.226562', '18.210938', '19.195312', '20.179688', '21.164062', '22.148438', '23.132812', '24.117188', '25.101562', '26.085938', '27.070312', '28.054688', '29.039062', '30.023438', '31.007812', '31.992188', '32.976562', '33.960938', '34.945312', '35.929688', '36.914062', '37.898438', '38.882812', '39.867188', '40.851562', '41.835938', '42.820312', '43.804688', '44.789062', '45.773438', '46.757812', '47.742188', '48.726562', '49.710938', '50.695312', '51.679688', '52.664062', '53.648438', '54.632812', '55.617188', '56.601562', '57.585938', '58.570312', '59.554688', '60.539062', '61.523438', '62.507812']


In [ ]:
counts = np.bincount(table["dischtime_day"])

In [ ]:
from pasteur.synth.hierarchy import RebalancedColumn

rc = RebalancedColumn(counts, col)
rc.height

63


In [ ]:
import math
from math import ceil, log

u = 1.3
fixed = [2, 4, 6, 8, 12]

height = col.height
max_domain = col.get_domain(0)

# Start by applying the fixed domain values
# If the fixed domain list goes higher than the domain of the attribute
# use the fixed list values that are lower, and append the maximum value at the end 
new_domain = []
for i, dom in enumerate(fixed):
    if dom >= max_domain:
        new_domain.append(max_domain)
        break
    else:
        new_domain.append(dom)

# If the fixed values don't go that high, continue by adding values that increase
# by u, yielding log(max_domain, u) levels
fixed_max_dom = fixed[-1]
if fixed_max_dom < max_domain:
    new_level_n = ceil(log(max_domain / fixed_max_dom, u))
    
    for i in range(1, new_level_n):
        dom = ceil(fixed_max_dom) * u ** i
        new_domain.append(int(dom))

    new_domain.append(max_domain)

new_domain, len(new_domain)

([2, 4, 6, 8, 12, 15, 20, 26, 34, 44, 57, 64], 12)
